This notebook trains a binary classifier on a dataset which contains movie reviews which are labelled as containing either *positive* or *negative* sentiment towards the movie.

First we will install *sklearn* which we will be using to do the machine learning.

In [1]:
pip install scikit-learn

Next we will install the dataset. We will use the IMDB sentiment analysis dataset available from the [huggingface datasets library](https://huggingface.co/datasets/imdb) and described in [Maas et al. 2011](https://aclanthology.org/P11-1015.pdf).

In [2]:
pip install datasets

Now let's load the IMDB training set. We will print out the last instance.

In [3]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")['train']
print(imdb_dataset[-1])

{'text': 'The story centers around Barry McKenzie who must go to England if he wishes to claim his inheritance. Being about the grossest Aussie shearer ever to set foot outside this great Nation of ours there is something of a culture clash and much fun and games ensue. The songs of Barry McKenzie(Barry Crocker) are highlights.', 'label': 1}


Let's convert the training data into the format expected by scikit-learn - a list of input vectors (documents) and a list of associated output labels.

In [4]:
train_data = []
train_data_labels = []
for item in imdb_dataset:
  train_data.append(item['text'])
  train_data_labels.append(item['label'])
print(train_data[-1])
print(train_data_labels[-1])

The story centers around Barry McKenzie who must go to England if he wishes to claim his inheritance. Being about the grossest Aussie shearer ever to set foot outside this great Nation of ours there is something of a culture clash and much fun and games ensue. The songs of Barry McKenzie(Barry Crocker) are highlights.
1


We'll use the CountVectorizer class to extract the words in each review as the features the algorithm will learn from. Each document is represented as a 1000 dimension vector of word  counts. Counts > 1 are clipped to 1. Stop words are removed.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='word',max_features=1000,lowercase=True,binary=True,stop_words='english')
features = vectorizer.fit_transform(train_data).toarray()

As a sanity check, let's check we have a 2-d array where each row is one of the 25,000 instances and each column is one of 16000 words or word bigrams. Print out the ngrams that will be used for classification.

In [6]:
print(features.shape)
print(vectorizer.get_feature_names_out())

(25000, 1000)
['10' '100' '15' '20' '30' '50' '80' '90' 'ability' 'able' 'absolutely'
 'accent' 'act' 'acted' 'acting' 'action' 'actor' 'actors' 'actress'
 'actual' 'actually' 'add' 'added' 'admit' 'adult' 'adventure' 'age' 'ago'
 'agree' 'air' 'alive' 'amazing' 'america' 'american' 'amusing'
 'animation' 'annoying' 'anti' 'apart' 'apparently' 'appeal' 'appear'
 'appearance' 'appears' 'appreciate' 'aren' 'art' 'aside' 'ask' 'aspect'
 'atmosphere' 'attempt' 'attempts' 'attention' 'audience' 'audiences'
 'average' 'avoid' 'away' 'awesome' 'awful' 'baby' 'background' 'bad'
 'badly' 'barely' 'based' 'basic' 'basically' 'battle' 'beautiful'
 'beautifully' 'beauty' 'begin' 'beginning' 'begins' 'believable'
 'believe' 'best' 'better' 'big' 'biggest' 'bit' 'bizarre' 'black' 'blood'
 'body' 'book' 'books' 'bored' 'boring' 'bought' 'box' 'boy' 'boys' 'br'
 'brain' 'break' 'brilliant' 'bring' 'brings' 'british' 'brother'
 'brothers' 'brought' 'budget' 'bunch' 'business' 'buy' 'called' 'came'
 'ca

Split the data into a training and validation (dev) set. We'll use the validation set to test our model. We'll use 75% of the data for training and 25% for testing.

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(features,train_data_labels,train_size=0.75,random_state=123)

We will use Multinomial Naive Bayes to do the classification. Create the model.

In [8]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

Train the model.

In [9]:
model = model.fit(X=X_train,y=y_train)

Test the model on the validation set.

In [10]:
y_pred = model.predict(X_val)

Now let's calculate the accuracy of the model's predictions on the validation set.

In [11]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_val,y_pred))

0.82784


In [12]:
import pickle
pickle.dump(model, open('ca337-nb1000-model.pkl', "wb"))
pickle.dump(vectorizer,open('ca337-nb1000-features.pkl','wb'))